<a href="https://colab.research.google.com/github/rename-z/Deep-Learning/blob/master/Zero%20to%20GANs/Linear%20Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Linear Regression with PyTorch**

We'll create a model that predicts crop yields for apples and oranges (*target variables*) by looking at the average temperature, rainfall and humidity (*input variables or features*) in a region. Here's the training data:

![linear-regression-training-data](https://i.imgur.com/6Ujttb4.png)

In a linear regression model, each target variable is estimated to be a weighted sum of the input variables, offset by some constant, known as a bias :

```
yield_apple  = w11 * temp + w12 * rainfall + w13 * humidity + b1
yield_orange = w21 * temp + w22 * rainfall + w23 * humidity + b2
```

In [1]:
import numpy as np
import torch

In [7]:
# Input (temperature, rainfall, humidity)
inputs = np.array([[73, 67, 43],
                   [91, 88, 64],
                   [87, 134, 58],
                   [102, 43, 37],
                   [69, 96, 70]], dtype='float32')
inputs

array([[ 73.,  67.,  43.],
       [ 91.,  88.,  64.],
       [ 87., 134.,  58.],
       [102.,  43.,  37.],
       [ 69.,  96.,  70.]], dtype=float32)

In [8]:
# Target (Apples, Oranges)
targets = np.array([[56, 70],
                    [81, 101],
                    [119, 133],
                    [22, 37],
                    [103, 119]], dtype='float32')
targets

array([[ 56.,  70.],
       [ 81., 101.],
       [119., 133.],
       [ 22.,  37.],
       [103., 119.]], dtype=float32)

In [9]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)
print(inputs)
print(targets)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]])
tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [10]:
# Weight and Biases
w = torch.randn(2, 3, requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[ 1.6768,  1.0685,  0.5240],
        [-0.1598,  1.0706,  0.6236]], requires_grad=True)
tensor([0.1355, 0.1685], requires_grad=True)


In [14]:
def model(x) :
  return x @ w.t() + b

In [15]:
preds = model(inputs)
print(preds)

tensor([[216.6655,  87.0457],
        [280.2908, 119.7468],
        [319.5904, 165.8922],
        [236.5058,  52.9746],
        [255.0924, 135.5694]], grad_fn=<AddBackward0>)


In [16]:
print(targets)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [17]:
def mse(t1, t2) :
  diff = t1 - t2
  return torch.sum(diff * diff) / diff.numel()

In [18]:
loss = mse(targets, preds)
loss

tensor(17716.5176, grad_fn=<DivBackward0>)

In [19]:
loss.backward()

In [20]:
print(w)
print(w.grad)

tensor([[ 1.6768,  1.0685,  0.5240],
        [-0.1598,  1.0706,  0.6236]], requires_grad=True)
tensor([[15937.8730, 15801.1816,  9976.1289],
        [ 1716.9213,  1895.3799,  1118.2845]])


In [21]:
w.grad.zero_()
b.grad.zero_()
print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [22]:
# 1.Generate predictions
preds = model(inputs)
preds

tensor([[216.6655,  87.0457],
        [280.2908, 119.7468],
        [319.5904, 165.8922],
        [236.5058,  52.9746],
        [255.0924, 135.5694]], grad_fn=<AddBackward0>)

In [23]:
# 2.Calculate the loss
loss = mse(preds, targets)
loss

tensor(17716.5176, grad_fn=<DivBackward0>)

In [24]:
# 3.Compute the gradients
loss.backward()
print(w.grad)
print(b.grad)

tensor([[15937.8730, 15801.1816,  9976.1289],
        [ 1716.9213,  1895.3799,  1118.2845]])
tensor([185.4290,  20.2457])


In [25]:
# 4 & 5.Adjust weights & reset gradients
with torch.no_grad():
  w -= w.grad * 1e-5
  b -= b.grad * 1e-5
  w.grad.zero_()
  b.grad.zero_()

In [26]:
print(w)
print(b)

tensor([[ 1.5175,  0.9105,  0.4242],
        [-0.1770,  1.0516,  0.6124]], requires_grad=True)
tensor([0.1337, 0.1683], requires_grad=True)


In [27]:
preds = model(inputs)
preds

tensor([[190.1525,  84.0413],
        [245.4957, 115.8006],
        [278.7628, 161.2098],
        [209.7616,  49.9943],
        [221.9410, 131.7822]], grad_fn=<AddBackward0>)

In [28]:
loss = mse(preds, targets)
loss

tensor(12152.5518, grad_fn=<DivBackward0>)

In [29]:
# Train for multiple epochs
for i in range(100) :
  preds = model(inputs)
  loss = mse(preds, targets)
  loss.backward()
  with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [30]:
preds = model(inputs)
loss = mse(preds, targets)
print(preds)
print(loss)

tensor([[ 63.6306,  70.4171],
        [ 82.5744,  98.0719],
        [107.4633, 138.6945],
        [ 58.1383,  37.2115],
        [ 80.9589, 114.5362]], grad_fn=<AddBackward0>)
tensor(204.6735, grad_fn=<DivBackward0>)


In [31]:
preds

tensor([[ 63.6306,  70.4171],
        [ 82.5744,  98.0719],
        [107.4633, 138.6945],
        [ 58.1383,  37.2115],
        [ 80.9589, 114.5362]], grad_fn=<AddBackward0>)

In [32]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])